# Анализ БД сервиса для чтения книг по подписке при помощи SQL

## Описание целей исследования и данных

**Описание проекта:**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель проекта:** 

Проанализировать информацию о книгах, издательствах, авторах, а также о пользовательских обзорах книг

**Задачи проекта:**
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**План работы:**

- Шаг 1: Исследование таблиц — выведение первых строк;
- Шаг 2: Создание по одному SQL-запросу для решения каждого задания;
- Шаг 3: Выведение результата каждого запроса в тетрадке;
- Шаг 4: Описание выводов по каждой из решённых задач.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

### books

In [2]:
query = '''

SELECT *
FROM books
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### authors

In [3]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### publishers

In [4]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### ratings

In [5]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### reviews

In [6]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Решение заданий

### 1. Cколько книг вышло после 1 января 2000 года

In [7]:
query = '''

SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


В базе доступна информация о 819 книгах, выпущенных после 1 января 2000 года.

### 2. Количество обзоров и средняя оценка для каждой книги

In [8]:
query = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS count_reviews, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id

LEFT JOIN  ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY count_reviews DESC

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


Всего в каталоге 1000 книг. Количество обзоров от 0 до 7.  Максимальное число обзоров на книгу Сумерки (7 ед.), при этом рейтинг у книги ниже 4.

### 3. Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query = '''
            SELECT b.publisher_id, p.publisher, COUNT(*) count_books
            FROM books b
            LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY COUNT(*) DESC
            LIMIT 1
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Издательство "Penguin Books" выпустило 42 книги толще 50 страниц. Это 4,2% от всех книг в базе.

### 4. Автор с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками)

In [10]:
query = '''
SELECT
    authors.author AS author,
    AVG(ratings.rating) AS avg_rating
    FROM ratings
    JOIN books ON books.book_id = ratings.book_id
    JOIN authors ON books.author_id = authors.author_id
    WHERE ratings.book_id IN (
       SELECT ratings.book_id
       FROM ratings
       GROUP BY ratings.book_id
       HAVING COUNT(ratings.book_id) >= 50)
    GROUP BY authors.author_id
    ORDER BY AVG(ratings.rating) DESC
    LIMIT 1'''
data = pd.io.sql.read_sql(query, con = engine)
data

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Самые популярный автор в базе со средним рейтингом в 4.29 баллов - это дуэт **Джоан Роулинг** (британской писательницы, автора популярной серии книг о Гарри Поттере) и американского иллюстратора **Мари Грандпре** (делала иллюстрации к обложкам и главам книг).

###  5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''
SELECT
    ROUND(AVG(sub_text_count.text_count)) AS avg_count_rv
FROM
    (SELECT
        COUNT(text) as text_count
    FROM 
        reviews
    WHERE username IN
        (SELECT username
        FROM 
            ratings
        GROUP BY
            username
        HAVING
            COUNT(ratings) > 48)
    GROUP BY 
            username
    ) AS sub_text_count  
'''

pd.io.sql.read_sql(query, con = engine)

,avg_count_rv
0,24.0


В среднем, активные пользователи (по условию - это пользователи, поставившие болеее 48-ти оценок) написали обзоры на 24 книги.

## Выводы

1. В базе доступна **информация о 819 книгах**, выпущенных после 1 января 2000 года;
2. Всего в каталоге 1000 книг. Количество обзоров от 0 до 7.  Максимальное число обзоров на книгу Сумерки (7 ед.), при этом рейтинг у книги ниже 4;
3. Издательство **"Penguin Books"** выпустило 42 книги толще 50 страниц. Это 4,2% от всех книг в базе;
4. Самые популярный автор в базе со средним рейтингом в 4.29 баллов - это **дуэт Джоан Роулинг** (британской писательницы, автора популярной серии книг о Гарри Поттере) **и американского иллюстратора Мари Грандпре** (делала иллюстрации к обложкам и главам книг);
5. В среднем, активные пользователи (по условию - это пользователи, поставившие болеее 48-ти оценок) написали обзоры на **24 книги.**